In [1]:
import os
os.chdir('../../scripts/external-data-download/')

In [2]:
OUTPUT_RELATIVE_PATH = "../../data/"
def create_path():
    """
        
    """
    path = ["external-raw-data", "curated/external-data", "raw/external-data"]
    for target_dir in path:
        if not os.path.exists(OUTPUT_RELATIVE_PATH + target_dir):
            os.makedirs(OUTPUT_RELATIVE_PATH + target_dir)
    print('Already Create Paths')

create_path()

Already Create Paths


In [3]:
import gdown
url_suburb_to_postcode = 'https://www.matthewproctor.com/Content/postcodes/australian_postcodes.csv'
output_convert = '../../data/external-raw-data/suburb-to-postcode.csv'
gdown.download(url_suburb_to_postcode, output_convert, quiet=False)

Downloading...
From: https://www.matthewproctor.com/Content/postcodes/australian_postcodes.csv
To: /Users/quzihanwu/Desktop/generic-real-estate-consulting-project-group-7/data/external-raw-data/suburb-to-postcode.csv
100%|██████████| 7.56M/7.56M [00:00<00:00, 16.8MB/s]


'../../data/external-raw-data/suburb-to-postcode.csv'

In [6]:
from utils import *

total_income()

http://house.speakingsame.com/suburbtop.php?sta=vic&cat=Median%20household%20income&name=Weekly%20income&page=0
9
['Rank', 'Suburb', 'Value']


/Users/quzihanwu/Desktop/generic-real-estate-consulting-project-group-7/scripts/external-data-download/utils.py:103: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_new, ignore_index=True)


http://house.speakingsame.com/suburbtop.php?sta=vic&cat=Median%20household%20income&name=Weekly%20income&page=1
9
['Rank', 'Suburb', 'Value']


/Users/quzihanwu/Desktop/generic-real-estate-consulting-project-group-7/scripts/external-data-download/utils.py:103: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_new, ignore_index=True)


http://house.speakingsame.com/suburbtop.php?sta=vic&cat=Median%20household%20income&name=Weekly%20income&page=2
9
['Rank', 'Suburb', 'Value']


...

http://house.speakingsame.com/suburbtop.php?sta=vic&cat=Median%20household%20income&name=Weekly%20income&page=12
9
['Rank', 'Suburb', 'Value']


/Users/quzihanwu/Desktop/generic-real-estate-consulting-project-group-7/scripts/external-data-download/utils.py:103: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_new, ignore_index=True)


http://house.speakingsame.com/suburbtop.php?sta=vic&cat=Median%20household%20income&name=Weekly%20income&page=13
9
['Rank', 'Suburb', 'Value']


/Users/quzihanwu/Desktop/generic-real-estate-consulting-project-group-7/scripts/external-data-download/utils.py:103: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_new, ignore_index=True)


()

In [2]:
from download import download

download.download_all()

/Users/quzihanwu/Desktop/generic-real-estate-consulting-project-group-7/scripts/external-data-download/download.py:217: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merge_select['Value'] = df_merge_select['Value'].str[1:]
/Users/quzihanwu/Desktop/generic-real-estate-consulting-project-group-7/scripts/external-data-download/download.py:219: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merge_select["Value"] = pd.to_numeric(df_merge_select["Value"])
